<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Alex-DEV/answer_combinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd
import json
import random
from itertools import chain, combinations
from datetime import datetime, timedelta

# Load questionnaires

In [139]:
dfs = []

for i in range(1, 6):
    url = f'https://raw.githubusercontent.com/alexk2206/tds_capstone/refs/heads/main/questionnaires/questionnaire{i}.json'
    df = pd.read_json(url)
    df['options'] = df['options'].apply(lambda x: ', '.join([opt['option'] for opt in x]))
    dfs.append(df)

all_questions = pd.concat(dfs, ignore_index=True)
print(f"all_questions shape: {all_questions.shape}")

all_questions shape: (25, 4)


# Define functions
- function for combination creation of MULTI_SELECT questions
- function for phone number creation
- function for date creation
- function for creation of note taking prompt
- function for processing different types of questions
- function to scale up the number of questions

In [140]:
def generate_combinations(options_list, max_size):
    return list(chain.from_iterable(combinations(options_list, r) for r in range(0, min(len(options_list), max_size) + 1)))


def generate_phone_number():
    phone_prefix = '01' + str(random.randint(100, 999)) + (str(random.randint(0, 9)) if random.random() < 0.5 else '')
    main_number = ''.join([str(random.randint(0, 9)) for _ in range(random.randint(6, 8))])
    phone_number = phone_prefix + main_number
    return [phone_number]


def generate_date(today=None):
    if today is None:
        today = datetime.today()

    random_days = random.randint(0, 13)
    random_date = today - timedelta(days=random_days)

    date = random_date.strftime('%Y-%m-%d')

    return [date]


def generate_notes():
    return ['Add additional information here']

In [141]:
def process_selections(row, max_size):
    question = row['question']
    options_list = row['options']
    question_type = row['type']
    expanded = []

    if question_type == 'MULTI_SELECT':
        options_combinations = generate_combinations(options_list, max_size=max_size)
        for combo in options_combinations:
            expanded.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': list(combo)})

    elif question_type == 'SINGLE_SELECT':
        for option in options_list:
            expanded.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': [option]})

    return expanded


def process_freetext(row):
    question = row['question']
    options_list = row['options']
    question_type = row['type']
    expanded = []

    if question_type == 'TEXT':
        expanded.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': generate_notes()})

    elif question_type == 'NUMBER':
        expanded.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': generate_phone_number()})

    elif question_type == 'DATE':
        expanded.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': generate_date()})

    return expanded

In [142]:
def adjust_question_amount(df, column, random_state):
    random.seed(random_state)
    def adjust_group(group):
        max_amount = random.randint(48, 64)

        if len(group) < max_amount:
            return group.sample(n=max_amount, replace=True, random_state=random_state)
        else:
            return group.sample(n=max_amount, random_state=random_state)

    return df.groupby(column, group_keys=False).apply(adjust_group).reset_index(drop=True)

# Apply functions on the dataset
- split dataset into selection questions and
- create intended answers for selection type questions
- scale up free text questions
- create intended answers for free text questions
- append dataset

In [143]:
selection_questions = all_questions[(all_questions['type'] == 'MULTI_SELECT') | (all_questions['type'] == 'SINGLE_SELECT')]
freetext_questions = all_questions[(all_questions['type'] != 'MULTI_SELECT') & (all_questions['type'] != 'SINGLE_SELECT')]

selection_counts = selection_questions['type'].value_counts()
freetext_counts = freetext_questions['type'].value_counts()

print(f'selection_questions shape: {selection_questions.shape}, counts per type:\n{selection_counts}')
print(f'freetext_questions shape: {freetext_questions.shape}, counts per type:\n{freetext_counts}')

selection_questions.loc[:, 'options'] = selection_questions['options'].str.split(', ').copy()
expanded_data = selection_questions.apply(lambda row: process_selections(row, max_size=6), axis=1).explode()
selection_intended_answers = pd.json_normalize(expanded_data)
selection_intended_answers.head(15)

selection_questions shape: (21, 4), counts per type:
type
SINGLE_SELECT    12
MULTI_SELECT      9
Name: count, dtype: int64
freetext_questions shape: (4, 4), counts per type:
type
TEXT      2
DATE      1
NUMBER    1
Name: count, dtype: int64


,question,type,options,intended_answer
0,Data processing consent,SINGLE_SELECT,"[Yes, No]",[Yes]
1,Data processing consent,SINGLE_SELECT,"[Yes, No]",[No]
2,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[End User]
3,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[Wholesaler]
4,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[Distributor]
5,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[Consultant]
6,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[Planner]
7,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[Architect]
8,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[R&D]
9,Products interested in,MULTI_SELECT,"[MY-SYSTEM, Notion, JTS, JS EcoLine, AKW100, A...",[]


In [144]:
selection_intended_answers_scaled = adjust_question_amount(selection_intended_answers, 'question', 1)

selection_intended_answers_scaled_counts = selection_intended_answers_scaled['type'].value_counts()
print(f'selection_questions shape: {selection_intended_answers_scaled.shape}, counts per type:\n{selection_intended_answers_scaled_counts}')

selection_questions shape: (1172, 4), counts per type:
type
SINGLE_SELECT    663
MULTI_SELECT     509
Name: count, dtype: int64


<ipython-input-142-ccd41743afd1>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(column, group_keys=False).apply(adjust_group).reset_index(drop=True)


In [145]:
freetext_questions_scaled = adjust_question_amount(freetext_questions, 'question', 1)

freetext_questions_scaled_counts = freetext_questions_scaled['type'].value_counts()
print(f'selection_questions shape: {freetext_questions_scaled.shape}, counts per type:\n{freetext_questions_scaled_counts}')

selection_questions shape: (209, 4), counts per type:
type
TEXT      102
NUMBER     56
DATE       51
Name: count, dtype: int64


<ipython-input-142-ccd41743afd1>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(column, group_keys=False).apply(adjust_group).reset_index(drop=True)


In [146]:
freetext_questions_scaled['options'] = freetext_questions_scaled['options'].str.split(', ')

expanded_rows = []

# Iteriere über alle Zeilen im DataFrame
for _, row in freetext_questions_scaled.iterrows():
    question = row['question']
    options_list = row['options']
    question_type = row['type']

    if question_type == 'NUMBER':
        expanded_rows.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer': generate_phone_number()})

    elif question_type == 'TEXT':
        expanded_rows.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer' : generate_notes()})

    elif question_type == 'DATE':
        expanded_rows.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer' : generate_date()})

    else:
        expanded_rows.append({'question': question, 'type': question_type, 'options': options_list, 'intended_answer' : options_list})

freetext_intended_answer_scaled = pd.DataFrame(expanded_rows)

In [147]:
combined_df = pd.concat([selection_intended_answers_scaled, freetext_intended_answer_scaled], ignore_index=True)

answer_combinations = combined_df.sample(frac=1, random_state=1).reset_index(drop=True)

print(f'answer_combinations shape: {answer_combinations.shape}')
answer_combinations.head(30)

answer_combinations shape: (1381, 4)


,question,type,options,intended_answer
0,When does the contact person wish to receive a...,MULTI_SELECT,"[1 week, 2 weeks, 3 weeks]","[2 weeks, 3 weeks]"
1,When does the contact person wish to receive a...,MULTI_SELECT,"[1 week, 2 weeks, 3 weeks]","[1 week, 2 weeks]"
2,Which language is wanted for communication?,SINGLE_SELECT,"[German, Italian, Japanese , English, Spanish]",[German]
3,Who to copy in follow up,MULTI_SELECT,"[Stephan Maier, Joachim Wagner, Erik Schneider...","[Erik Schneider, Angelina Haug, Jessica Hanke,..."
4,Customer group,SINGLE_SELECT,"[End User, Wholesaler, Distributor, Consultant...",[End User]
5,What is the type of contact?,MULTI_SELECT,"[Existing customer, Supplier, New customer / P...","[New customer / Prospect, Press / media, Compe..."
6,What phone number can we use for contact?,NUMBER,[phone number],[011318636083]
7,When do you wish to receive a follow-up?,DATE,[Date],[2025-01-14]
8,Next steps,SINGLE_SELECT,"[Offer, Meeting, Call]",[Offer]
9,What is the contact person interested in?,MULTI_SELECT,"[100 Additive Manufacturing, 200 Automation, 3...","[300 Advanced Manufacturing, Others]"


In [148]:
answer_combinations.to_json('answer_combinations.json', orient='records')